In [1]:
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_VISIBLE_DEVICES=1


In [2]:
import os
from PIL import Image

import time
import pickle
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import torch
from torchvision import datasets, transforms

print(f"Torch: {torch.__version__}")
device = 'cuda'
use_cuda = torch.cuda.is_available()
print(use_cuda)

Torch: 2.0.1+cu118
True


In [3]:
torch.cuda.get_device_name()

'NVIDIA GeForce RTX 2080 Ti'

In [4]:
PATH='/home/avsavchenko/src/face-emotion-recognition/models/affectnet_emotions/enet_b0_8_best_vgaf.pt'
IMG_SIZE=224


test_transforms = transforms.Compose(
    [
        transforms.Resize((IMG_SIZE,IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    ]
)
np_transforms = transforms.Compose(
    [
        transforms.ToPILImage(None),
        transforms.Resize((IMG_SIZE,IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    ]
)

feature_extractor_model = torch.load(PATH)

In [5]:
classifier_weights=feature_extractor_model.classifier[0].weight.cpu().data.numpy()
classifier_bias=feature_extractor_model.classifier[0].bias.cpu().data.numpy()

In [6]:
feature_extractor_model.classifier=torch.nn.Identity()
feature_extractor_model=feature_extractor_model.to(device)
feature_extractor_model.eval();

# Get features

In [7]:
DATA_DIR = '/home/HDD6TB/datasets/emotions/ABAW/ABAW_6/6th_ABAW_Annotations'

In [32]:
dirnames = ['cropped','cropped_aligned/cropped_aligned_new_50_vids']
# dirnames = ['cropped_aligned/cropped_aligned','cropped_aligned/cropped_aligned_new_50_vids']

In [33]:
l = 0
for d in dirnames:
    dirpath=os.path.join(DATA_DIR, d)
    for folder in tqdm(os.listdir(dirpath)):
        if folder=='.DS_Store': continue
        for filename in os.listdir(os.path.join(dirpath, folder)):
            fn, ext = os.path.splitext(os.path.basename(filename))
            if ext.lower()=='.jpg':
                l+=1
            
l

  0%|          | 0/594 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

3227429

In [8]:
def get_probab(features):
    x=np.dot(features,np.transpose(classifier_weights))+classifier_bias
    return x

In [9]:
print(test_transforms)
img_names=[]
X_global_features=[]
imgs=[]

MODEL2FEATURES = 'enet_b0_8_best_vgaf_cropped.pickle' 

Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)


In [10]:
len(img_names), len(X_global_features)

(0, 0)

In [ ]:
for d in dirnames:
    data_dir=os.path.join(DATA_DIR, d)
    print(data_dir)
    
    for _, filename in enumerate(tqdm(os.listdir(data_dir))):
        
        frames_dir = os.path.join(data_dir, filename)
        if '.DS_Store'==filename:
            continue
        
#         if filename in num_files.keys():
#             if len(os.listdir(frames_dir)) == num_files[filename]:
#                 continue
            
        for img_name in os.listdir(frames_dir):
            
            if img_name.lower().endswith('.jpg'):
                    
                img = Image.open(os.path.join(frames_dir,img_name))
                img_tensor = test_transforms(img)

                if img.size:

                    img_names.append(filename+'/'+img_name)
                    imgs.append(img_tensor)

                    if len(imgs)>=64:
                        features = feature_extractor_model(torch.stack(imgs, dim=0).to(device))
                        features=features.data.cpu().numpy()

                        if len(X_global_features)==0:
                            X_global_features=features
                        else:
                            X_global_features=np.concatenate((X_global_features,features),axis=0)
                        imgs=[]


if len(imgs)>0:
    features = feature_extractor_model(torch.stack(imgs, dim=0).to(device))
    features=features.data.cpu().numpy()

    if len(X_global_features)==0:
        X_global_features=features
    else:
        X_global_features=np.concatenate((X_global_features,features),axis=0)

    imgs=[]

print(X_global_features.shape)

In [12]:
X_scores=get_probab(X_global_features)                
filename2featuresAll={img_name:(global_features,scores) for img_name,global_features,scores in zip(img_names,X_global_features,X_scores)}

with open(MODEL2FEATURES, 'wb') as handle:
    pickle.dump(filename2featuresAll, handle, protocol=pickle.HIGHEST_PROTOCOL)